In [2]:
from sympy import *
from sympy.physics.quantum import TensorProduct
from sympy.physics.matrices import msigma

In [3]:
def U(theta, phi, gamma):
    return Matrix(
        [
            [cos(theta / 2), -exp(1j * gamma) * sin(theta / 2)],
            [exp(1j * phi) * sin(theta / 2), exp(1j * (phi + gamma)) * cos(theta / 2)],
        ]
    )


def controlled_gate(U, switch=False):
    if not switch:
        return Matrix(BlockDiagMatrix(eye(2), U))
    else:
        return Matrix([[1, 0, 0, 0], [0, U[0, 0], 0, U[0, 1]], [0, 0, 1, 0], [0, U[1, 0], 0, U[1, 1]]])


CX = Matrix(BlockDiagMatrix(eye(2), Matrix([[0, 1], [1, 0]])))
CXT2 = Matrix([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]])

alpha, beta = symbols("alpha, beta")
theta1, phi1, gamma1 = symbols("theta1, phi1, gamma1")
theta2, phi2, gamma2 = symbols("theta2, phi2, gamma2")
theta3, phi3, gamma3 = symbols("theta3, phi3, gamma3")

In [4]:
U(theta1, phi1, gamma1)

Matrix([
[                cos(theta1/2),         -exp(1.0*I*gamma1)*sin(theta1/2)],
[exp(1.0*I*phi1)*sin(theta1/2), exp(1.0*I*(gamma1 + phi1))*cos(theta1/2)]])

In [5]:
D, e, t = symbols('Delta, varepsilon, t')

T = exp(1j * D * msigma(3) * t / 4) * exp(1j * e * msigma(2) * t / 2) * exp(1j * D * msigma(3) * t / 4)#

sol = solve(T - U(theta1, phi1, gamma1) * exp(1j * phi2), theta1, phi1, gamma1, phi2, dict=True)

In [16]:
sol[0]

{phi2: I*(0.346573590279973 - log((exp(I*t*varepsilon) + 1.0)*sqrt(1/(cos(t*varepsilon) + 1.0))*exp(0.5*I*t*(Delta - varepsilon)))),
 gamma1: -I*log(-exp(-0.5*I*Delta*t)),
 phi1: -I*log(-exp(-0.5*I*Delta*t)),
 theta1: 2.0*atan(tan(0.5*t*varepsilon))}

In [7]:
simplify(U(theta1, phi1, gamma1).subs(sol[0]))

Matrix([
[                            cos(1.0*atan(tan(0.5*t*varepsilon))), -(-exp(-0.5*I*Delta*t))**1.0*sin(1.0*atan(tan(0.5*t*varepsilon)))],
[(-exp(-0.5*I*Delta*t))**1.0*sin(1.0*atan(tan(0.5*t*varepsilon))),  (-exp(-0.5*I*Delta*t))**2.0*cos(1.0*atan(tan(0.5*t*varepsilon)))]])

In [8]:
init = TensorProduct(Matrix([alpha, beta]), Matrix([1, 0]))
init

Matrix([
[alpha],
[    0],
[ beta],
[    0]])

In [9]:
# Loss
CU = Matrix(BlockDiagMatrix(eye(2), U(theta1, 0, 0)))
CUT2 = controlled_gate(U(theta1, phi1, gamma1), True)

circuit1 = CXT2 * CU
circuit1


Matrix([
[1, 0,             0,              0],
[0, 0, sin(theta1/2),  cos(theta1/2)],
[0, 0, cos(theta1/2), -sin(theta1/2)],
[0, 1,             0,              0]])

In [10]:
circuit1 * init

Matrix([
[             alpha],
[beta*sin(theta1/2)],
[beta*cos(theta1/2)],
[                 0]])

In [11]:
## Loss with fewer two qubit gates

L = (
    TensorProduct(U(0, 0, -pi / 2), U(0, 0, -pi / 2))
    * CXT2
    * TensorProduct(U(pi / 2, -pi / 2, 0), U(-theta1 / 2, pi, pi / 2))
    * CXT2
    * TensorProduct(U(pi / 2, -pi, 0), U(-theta1 / 2, -pi / 2, pi))
)
L = simplify(L)
L


Matrix([
[1, 0,             0,              0],
[0, 0, sin(theta1/2),  cos(theta1/2)],
[0, 0, cos(theta1/2), -sin(theta1/2)],
[0, 1,             0,              0]])

In [12]:
L * init

Matrix([
[             alpha],
[beta*sin(theta1/2)],
[beta*cos(theta1/2)],
[                 0]])

In [13]:
# Gain

U1 = TensorProduct(U(-pi, 0, 0), eye(2))
CU1 = controlled_gate(U(theta2, 0, 0), True)
U2 = TensorProduct(U(pi, 0, 0), eye(2))


# circuit = U2 * CX * CU1 * CX * U1
circuit2 = U2 * CX * CU1 * CX * U1
circuit2

Matrix([
[cos(theta2/2), 0, 0, -sin(theta2/2)],
[            0, 1, 0,              0],
[            0, 0, 1,              0],
[sin(theta2/2), 0, 0,  cos(theta2/2)]])

In [14]:
circuit2 * init

Matrix([
[alpha*cos(theta2/2)],
[                  0],
[               beta],
[alpha*sin(theta2/2)]])